<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/keras/cnn/darknet53_on_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**test gpu**

In [ ]:
!nvidia-smi

**imports**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# keras and tf
import tensorflow as tf
import keras

# models
from keras.models import Sequential
from keras.models import Model

# backend
from keras import optimizers, metrics, models
import keras.backend as K

# layers
from keras.layers import Input, add, Conv2D, Flatten, Dense, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

# optimizers
from keras.optimizers import SGD

# regularizers
from keras.regularizers import l2

# training
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# save
import h5py

# keras aplications
from keras.applications import DenseNet201, DenseNet169, InceptionResNetV2, ResNet152V2, InceptionV3, DenseNet121, Xception, MobileNet, VGG19


# other libs
import math
import pickle
import os

**connect to the drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**set options**

In [ ]:
# colab cant handle image size of 516
image_size = 224 
batch_size = 32
nr_classes = 106
nr_ephocs = 50
model_path = "/content/drive/My Drive/darknet53_106_class_model.hdf5"

**generate data**

In [ ]:
train_datagen = ImageDataGenerator(
        # rotation -40 40
        rotation_range=40,

        # shift images and fill pixels with nearest
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',

        horizontal_flip=True,
        zoom_range=0.3,

        shear_range=0.2,

        brightness_range=[0.1, 0.5],
        channel_shift_range=0.5,

        # normalization 
        featurewise_center=True,
        featurewise_std_normalization=True,
        # validation_split=0.2,
        )

valid_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

train = train_datagen.flow_from_directory("/content/drive/My Drive/clean_data/train", target_size=(image_size,image_size), batch_size=batch_size)
valid = valid_datagen.flow_from_directory("/content/drive/My Drive/clean_data/test", target_size=(image_size,image_size), batch_size=batch_size)

**build model**     [github link of author](https://github.com/xiaochus/YOLOv3/blob/master/model/darknet53.py)

In [ ]:
"""Darknet-53 for yolo v3.
"""
def conv2d_unit(x, filters, kernels, strides=1):
    """Convolution Unit
    This function defines a 2D convolution operation with BN and LeakyReLU.
    # Arguments
        x: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernels: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and
            height. Can be a single integer to specify the same value for
            all spatial dimensions.
    # Returns
            Output tensor.
    """
    x = Conv2D(filters, kernels,
               padding='same',
               strides=strides,
               activation='linear',
               kernel_regularizer=l2(5e-4))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    return x


def residual_block(inputs, filters):
    """Residual Block
    This function defines a 2D convolution operation with BN and LeakyReLU.
    # Arguments
        x: Tensor, input tensor of residual block.
        kernels: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
    # Returns
        Output tensor.
    """
    x = conv2d_unit(inputs, filters, (1, 1))
    x = conv2d_unit(x, 2 * filters, (3, 3))
    x = add([inputs, x])
    x = Activation('linear')(x)

    return x


def stack_residual_block(inputs, filters, n):
    """Stacked residual Block
    """
    x = residual_block(inputs, filters)

    for i in range(n - 1):
        x = residual_block(x, filters)

    return x


def darknet_base(inputs):
    """Darknet-53 base model.
    """

    x = conv2d_unit(inputs, 32, (3, 3))

    x = conv2d_unit(x, 64, (3, 3), strides=2)
    x = stack_residual_block(x, 32, n=1)

    x = conv2d_unit(x, 128, (3, 3), strides=2)
    x = stack_residual_block(x, 64, n=2)

    x = conv2d_unit(x, 256, (3, 3), strides=2)
    x = stack_residual_block(x, 128, n=8)

    x = conv2d_unit(x, 512, (3, 3), strides=2)
    x = stack_residual_block(x, 256, n=8)

    x = conv2d_unit(x, 1024, (3, 3), strides=2)
    x = stack_residual_block(x, 512, n=4)

    return x


def darknet():
    """Darknet-53 classifier.
    """
    inputs = Input(shape=(image_size, image_size, 3))
    x = darknet_base(inputs)

    x = GlobalAveragePooling2D()(x)
    x = Dense(nr_classes, activation='softmax')(x)

    model = Model(inputs, x)

    return model

**instantiate model**

In [ ]:
model = darknet()
model.summary()

**or load model**

In [ ]:
model = keras.models.load_model(model_path)

**set optimizer**

In [ ]:
opt = SGD(lr=0.001, momentum=0.9)

**compile model**

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

**set checkpointer**

In [ ]:
checkpointer = ModelCheckpoint(filepath= model_path, monitor='val_loss', save_best_only=True)

**train**

In [ ]:
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid, callbacks=[checkpointer])